In [5]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

Extração

In [15]:
import pandas as pd

df = pd.read_csv("sdw2023.csv")
user_ids = df['UserID'].tolist()
print(user_ids)

[5856, 5857, 5858]


In [17]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 5856,
    "name": "Jax",
    "account": {
      "id": 6205,
      "number": "4508-7",
      "agency": "0008",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 5690,
      "number": "**** **** **** 4509",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5857,
    "name": "Sektor",
    "account": {
      "id": 6206,
      "number": "4509-7",
      "agency": "0009",
      "balance": 0.0,
      "limit": 600.0
    },
    "card": {
      "id": 5691,
      "number": "**** **** **** 4519",
      "limit": 1200.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 5858,
    "name": "Syrax",
    "account": {
      "id": 6207,
      "number": "4503-7",
      "agency": "0003",
      "balance": 0.0,
      "limit": 700.0
    },
    "card": {
      "id": 5692,
      "number": "**** **** **** 4722",
      "limit": 1400.0
    },
    "features": [],
    "news": []
  }
]


In [18]:
!pip install openai

In [19]:
openai_api_key = '#'

Transformação

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Você é um especialista em marketing bancário."},
        {"role": "user", "content": f"Crie uma mensagem para{user['name']} sobre a importância dos investimentos (max de 100 caracteres)"},
        #{"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        #{"role": "user", "content": "Where was it played?"}
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })



Carregamento

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}")